In [1]:
import pandas as pd
import pickle
from scipy.spatial.distance import euclidean
import datetime as dt
import numpy as np

In [2]:
# Loading the datasets

base_path = 'C:/Users/Mikel/Desktop/Archivos/Estudios/Ciencia de Datos/TFM/Tercera iteración'
air_data = base_path + '/air_quality_dict.pickle'
nearest_traffic_data = base_path + '/nearest_dict.pickle'
nearest_air_data = base_path + '/nearest_air_dict.pickle'
geo_data = base_path + '/geo_dict.pickle'
traffic_data = base_path + '/traffic_dict.pickle'

with open(air_data, 'rb') as handle:
    air_dict = pickle.load(handle)

with open(nearest_traffic_data, 'rb') as handle:
    nearest_traffic_dict = pickle.load(handle)
    
with open(nearest_air_data, 'rb') as handle:
    nearest_air_dict = pickle.load(handle)
    
with open(geo_data, 'rb') as handle:
    geo_dict = pickle.load(handle)
    
with open(traffic_data, 'rb') as handle:
    traffic_dict = pickle.load(handle)
    
meteo_path ='C:/Users/Mikel/Desktop/Archivos/Estudios/Ciencia de Datos/TFM/Tercera iteración/meteo.csv'
meteo_df = pd.read_csv(meteo_path)

In [3]:
# We create a dict with the desired daterange

air_dict_dates = {}
for key, df in air_dict.items():
    df = df.loc['2021-07-01':'2022-06-30']
    air_dict_dates[key] = df
    
# We filter dates from the meteo dataframe to fit the desired daterange
    
meteo_df = meteo_df.set_index('time')
meteo_dates = meteo_df.loc['2021-07-01':'2022-07-01']

In [4]:
# We join traffic data with air quality data

complete_dict = {}
for key, df in traffic_dict.items():
    air_station = nearest_traffic_dict[key]
    air_df = air_dict_dates[air_station]
    df = pd.concat([df, air_df.reindex(df.index)], axis = 1)
    geo_df = geo_dict[key]
    for column in geo_df.columns:
        col_ = geo_df[column].values.tolist()
        col = col_ * len(df.index)
        df[column] = col
    for column in meteo_dates.columns:
        df[column] = meteo_dates[column].values
    complete_dict[key] = df

In [5]:
# Addition of festivities, weekends and school holidays

# Weekends

a_station = complete_dict[10498]
a_station_i = a_station.index.to_series()
weekdays = a_station_i.dt.dayofweek
is_weekend = weekdays > 4
is_weekend = is_weekend.astype(int)

# School holidays & bank holidays

index_f = a_station_i.dt.strftime('%Y-%m-%d')
fechas_list = index_f.tolist()
ver_2021 = pd.date_range('20210701','20210906')
hisp_2021 = pd.date_range('20211009','20211012')
sant_2021 = pd.date_range('20211030','20211101')
cons_2021 = pd.date_range('20211204','20211208')
nav_2021 = pd.date_range('20211223','20211231')
nav_2022 = pd.date_range('20220101','20220110')
carn_2022 = pd.date_range('20220225','20220228')
ss_2022 = pd.date_range('20220408','20220418')
mad_2022 = pd.date_range('20220430','20220502')
ver_2022 = pd.date_range('20220627','20220630')

binaries = []
dates_list = []
holidays = [ver_2021, hisp_2021, sant_2021, cons_2021, nav_2021, nav_2022, carn_2022, ss_2022, mad_2022, ver_2022]
for list_ in holidays:
    list_ = list_.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
    for item in list_:
        dates_list.append(item)
for item in fechas_list:
    if item in dates_list:
        binaries.append(1)
    elif item not in dates_list:
        binaries.append(0)
        
# Combination of both infos

holiday_weekends = []
weekend_list = is_weekend.values.tolist()
for i in range(0, len(weekend_list)):
    if weekend_list[i] == 1 or binaries[i] == 1:
        holiday_weekends.append(1)
    elif weekend_list[i] == 0 or binaries[i] == 0:
        holiday_weekends.append(0)

# Addition to dict

complete_dict_ = {}
for key, df in complete_dict.items():
    df['holiday'] = holiday_weekends
    complete_dict_[key] = df
    

In [6]:
# We get the utm coordinates of the stations

estaciones_dataset = complete_dict_.keys()
estaciones_trafico = pd.read_csv('C:/Users/Mikel/Desktop/estaciones_chamberi.csv')
mask = estaciones_trafico['id'].isin(estaciones_dataset)
estaciones_trafico = estaciones_trafico[mask]
estaciones_trafico = estaciones_trafico[['id', 'utm_x', 'utm_y']]

In [7]:
# We get the stations following a series of buffers

euclidean_dict = {}
estaciones = estaciones_trafico['id'].tolist()
trafico_x = estaciones_trafico['utm_x'].tolist()
trafico_y = estaciones_trafico['utm_y'].tolist()
trafico_coords = list(zip(trafico_x, trafico_y))
for i in range(0, len(trafico_coords)):
        key = estaciones[i]
        station_dict = {}
        list_25 = []
        list_50 = []
        list_100 = []
        list_200 = []
        list_500 = []
        list_1000 = []
        for j in range(0, len(trafico_coords)):
            dist = euclidean(trafico_coords[i], trafico_coords[j])
            if dist > 0 and dist < 25:
                list_25.append(estaciones[j])
            if dist > 0 and dist < 50:
                list_50.append(estaciones[j])
            if dist > 0 and dist < 100:
                list_100.append(estaciones[j])
            if dist > 0 and dist < 200:
                list_200.append(estaciones[j])
            if dist > 0 and dist < 500:
                list_500.append(estaciones[j])
            if dist > 0 and dist < 1000:
                list_1000.append(estaciones[j])
        station_dict[25] = list_25
        station_dict[50] = list_50
        station_dict[100] = list_100
        station_dict[200] = list_200
        station_dict[500] = list_500
        station_dict[1000] = list_1000
        euclidean_dict[key] = station_dict

In [8]:
# We create the final complete dict

complete_dict_buf = {}
for key, df in complete_dict_.items():
    buffer_dict = euclidean_dict[key]
    for k, l in buffer_dict.items():
        if l:
            for item in l:
                colname = str(k) + '_' + str(item)
                traffic_df = complete_dict[item]['intensidad']
                traffic_df = traffic_df.rename(colname)
                df = pd.concat([df, traffic_df.reindex(df.index)], axis = 1)
    complete_dict_buf[key] = df

In [9]:
# We export the result to a csv file

base_path = 'C:/Users/Mikel/Desktop/Archivos/Estudios/Ciencia de Datos/TFM/Tercera iteración/dataset'
for key, df in complete_dict_buf.items():
    filename = '/dataset_' + str(key) + '.csv'
    filepath = base_path + filename
    df.to_csv(filepath)

In [10]:
# We get the keys of the final stations

keys = []
for key in complete_dict_buf.keys():
    keys.append(key)
keys_df = pd.DataFrame(keys)

In [11]:
# We export the keys to a separate df

file_path = 'C:/Users/Mikel/Desktop/Archivos/Estudios/Ciencia de Datos/TFM/Tercera iteración/keys_df.csv'
keys_df.to_csv(file_path)

In [12]:
# Exporting the datasets

with open('C:/Users/Mikel/Desktop/Archivos/Estudios/Ciencia de Datos/TFM/Tercera iteración/complete_dict_buf.pickle' \
          , 'wb') as handle:
    pickle.dump(complete_dict_buf, handle, protocol=pickle.HIGHEST_PROTOCOL)